<img align="right" width="200" height="200" src="https://static.wikia.nocookie.net/lego/images/2/23/PPG_logo.png"> 

# <span style="color:#EF7C8E">Introduction 3 - Machine Learning 3 Assignment - Machine Learning </span>
 


## <span style="color:#55BB99"> By: Wejdan Al-Ahmadi</span>
<hr style="border:2px solid gray"> </hr>

### <span style="color:#EF7C8E">1. Build an Evaluation function: </span>

Build a function that takes a vector of predictions using your heuristic and a vector of realizations (the correct values) from the data set and calculate:
* Precision (Classification "RAIN" column)
* Recall  (Classification "RAIN" column)
* SSE Cost of your prediction (Regression "PRCP" column)



In [1]:
#here is an example of how to build and populate a hurestic model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import time
import datetime
from matplotlib.pyplot import figure
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, recall_score, precision_score

import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

#### <span style="color:#55BB99"> 1) Data Import and Cleaning:</span>

In [2]:
# Load Data
df = pd.read_csv('./data/seattle_weather_1948-2017.csv')

def RAIN_INSERTION(cols):
    """
    Insert False where NaN values are present
    """
    RAIN=cols[0]
    if pd.isnull(RAIN):
        return False
    else:
        return RAIN
    
def PRCP_INSERTION(col):
    """
    Insert the Mean of PRCP where NaN values are present
    """
    PRCP=col[0]
    if pd.isnull(PRCP):
        return df['PRCP'].mean()
    else:
        return PRCP
    
# Apply the functions
df['RAIN']=df[['RAIN']].apply(RAIN_INSERTION,axis=1)
df['PRCP']=df[['PRCP']].apply(PRCP_INSERTION,axis=1)

# First quartile (Q1)
Q1 = np.percentile(df['TMAX'], 25, interpolation = 'midpoint')
  
# Third quartile (Q3)
Q3 = np.percentile(df['TMAX'], 75, interpolation = 'midpoint')
  
# Interquaritle range (IQR)
IQR = Q3 - Q1

# lower bound outliers --> Q1 - 1.5(IQR)
# higher bound outliers --> Q3 + 1.5 (IQR)

print(Q3+ 1.5*(IQR))

#Dropping the outliers from TMIN column
df=df.drop(df[df['TMIN']<17 ].index)
#Dropping the outliers from TMAX columns i.e. the value more than 100
df=df.drop(df[(df['TMAX']>97.5) | (df['TMAX']< 21.5)].index)
#Dropping the outliers from PRCP columns i.e. the value more than 0.275
df=df.drop(df[(df['PRCP']>0.25) | (df['PRCP']< -0.15) ].index)

# Reset index and drop index column
df = df.reset_index().drop("index", axis=1)

97.5


#### <span style="color:#55BB99"> 2) Hueristic and Confusion Matrix function for Rain and PRCP:</span>

In [3]:
rain_pred_df = df.copy()

In [4]:
# x --> Tomorrow
# x-1 --> Today
# x-2 --> Yesterday

def heuristic_rain(df):
    
    """
    Simple heuristic:
    
    If it rained yesterday or the day before yesterday
    
    then predict rain else predict no rain
    
    Frist two rows are predicted false be default
    """
    
    preds = []
    for x in range(len(df)):
        if x <2:
            preds.append(False)
        else:
            if (df.iloc[x-1]["RAIN"] == True):
                if(df.iloc[x]["TMAX"]<=65) & (df.iloc[x]["PRCP"] >=0.01)&(df.iloc[x]["TMIN"]<=45):
                    preds.append(True)
                else: 
                    preds.append(False)  
            elif (df.iloc[x-2]["RAIN"] == True):
                if (df.iloc[x]["TMAX"]<=65) & (df.iloc[x]["PRCP"] >=0.01)&(df.iloc[x]["TMIN"]<=45):
                    preds.append(True)
                else:
                    preds.append(False)
            elif (df.iloc[x-3]["RAIN"] == True):
                if (df.iloc[x]["TMAX"]<=65) & (df.iloc[x]["PRCP"] >=0.01)&(df.iloc[x]["TMIN"]<=45):
                    preds.append(True)
                else:
                    preds.append(False)

            else: 
                preds.append(False)
    return preds



In [5]:
def calc_confuse_rain(true_c, pred_c):
    
    "Calculate all possible results of a confusion matrix"

    # Hold all possible values and set to zero
    FP = np.zeros(len(true_c))
    TP = np.zeros(len(true_c))
    FN = np.zeros(len(true_c))
    TN = np.zeros(len(true_c))
    
    for x in range(len(true_c)):
        
        # True Positive
        if (true_c[x] == True) & (pred_c[x] == True):
            TP[x] = 1
        # True Negative
        elif (true_c[x] == False) & (pred_c[x] == False):
            TN[x] = 1
        # False Negative
        elif (true_c[x] == True) & (pred_c[x] == False):
            FN[x] = 1
        # False Positive
        else:
            FP[x] = 1
    
    return FP, TP, FN, TN

In [6]:
# get a copy
prcp_pred_df = df.copy()

"""
This function predicts the PRCP using the mean of the column 
"""
def heuristic_prcp_mean(df):
    preds = np.zeros(len(df))
    for x in range(len(df)):
        preds[x]=round(df.PRCP.mean(), 2)
    return preds

# Apply Heuristic
prcp_pred_df["PRCP_mean"] = heuristic_prcp_mean(prcp_pred_df)

#### <span style="color:#55BB99"> 3) RSS, Recall and Precision funcs:</span>

In [7]:
# A Sum of Squared Error Function
def rss(y_true, y_pred):
    squared_delta=(y_true-y_pred)**2
    rss=sum(squared_delta)
    return rss

# A Function to Calculate Recall
def recall(TP, FN):
    return(sum(TP)/sum(TP+FN))

# A Function to Calculate Precision
def precision(TP, FP):
    bottom=sum(TP+FP)
    top = sum(TP)
    return top/bottom 

#### <span style="color:#55BB99"> 4) Evaluation function:</span>

In [8]:
def evaluation(true_r=pd.Series, pred_r=pd.Series, true_c=pd.Series, pred_c=pd.Series):   
    regression=""                                                                   # string to store regression output
    if(len(true_r)!=0) & (len(pred_r)!=0):                                          # check if regression data passed in
        regression = f"The SSE is: {rss(true_r, pred_r)}"                           # format the regression var and call rss
    classification=""                                                               # string to store classification output
    if(len(true_c)!=0) & (len(pred_c)!=0):                                          # check if classification data passed in
        FP,TP,FN,FN = calc_confuse_rain(rain_pred_df["RAIN"],rain_pred_df["preds"]) # calc confusion matrix
        prec = precision(TP, FP)                                                    # get the precision
        rec = recall(TP, FN)                                                        # get the recall
        classification= f"The Precisin is: {prec} and the Recall is: {rec}"         # format the classification var with precision and recall
    return print("\n"+regression+"\n"+classification+"\n")                          # return a print with both


In [9]:
# Apply Heuristic
rain_pred_df["preds"] = heuristic_rain(rain_pred_df)

# print the evaluation for both regression and classification
evaluation(prcp_pred_df["PRCP"],prcp_pred_df["PRCP_mean"],rain_pred_df["RAIN"],rain_pred_df["preds"])

# initialize fake dummy data
data = pd.DataFrame(columns = [1,2,3,4])

# if you only wanna pass in regression: 
evaluation(prcp_pred_df["PRCP"],prcp_pred_df["PRCP_mean"],data,data)
# if you only wanna pass in classification: 
evaluation(data,data,rain_pred_df["RAIN"],rain_pred_df["preds"])


The SSE is: 76.90262920701885
The Precisin is: 1.0 and the Recall is: 0.20859362268455647


The SSE is: 76.90262920701885




The Precisin is: 1.0 and the Recall is: 0.20859362268455647



In [10]:
# Here I seperated the function above into 2 different functions
def evaluation_R(true_r=pd.Series, pred_r=pd.Series):   
    regression=""                                                   # string to store regression output
    if(len(true_r)!=0) & (len(pred_r)!=0):                          # check if regression data passed in
        regression = f"The SSE is: {rss(true_r, pred_r)}"           # format the regression var and call rss
    return print("\n"+regression+"\n" )                             # return a print with both

def evaluation_C(true_c=pd.Series, pred_c=pd.Series):   
    classification=""                                                               # string to store classification output
    if(len(true_c)!=0) & (len(pred_c)!=0):                                          # check if classification data passed in
        FP,TP,FN,FN = calc_confuse_rain(rain_pred_df["RAIN"],rain_pred_df["preds"]) # calc confusion matrix
        prec = precision(TP, FP)                                                    # get the precision
        rec = recall(TP, FN)                                                        # get the recall
        classification= f"The Precisin is: {prec} and the Recall is: {rec}"         # format the classification var with precision and recall
    return print("\n"+classification+"\n")                                          # return a print with both

# Call regression evaluation: 
evaluation_R(prcp_pred_df["PRCP"],prcp_pred_df["PRCP_mean"])
# Call classification evaluation: 
evaluation_C(rain_pred_df["RAIN"],rain_pred_df["preds"])


The SSE is: 76.90262920701885


The Precisin is: 1.0 and the Recall is: 0.20859362268455647



### <span style="color:#EF7C8E">2. Training/Testing: </span>

* Separate your data set into training and testing. (80/20 split)
* Calculate the Precision and Recall for the classification heuristic you made on Sunday
* Calculate the MSE, MAE, or SSE for the regression heuristic you made Monday.
* Save your results and repeat the process 5 times.
* Once you have repeated steps 1-4 5 times and saved the results, calculate the average score from your saved results
* Submit your notebook to the Learn Platform when you have finished.

#### <span style="color:#55BB99"> 1) Initializing MSE, MAE:</span>

In [11]:
# A Mean Squared Error Function
def mse(y_true, y_pred):
    squared_delta=(y_true-y_pred)**2
    mse=sum(squared_delta)/len(y_true)
    return mse

# A Mean Absolute Error Function
def mae(y_true, y_pred):
    abs_delta=abs(y_true-y_pred)
    mae=sum(abs_delta)/len(y_true)
    return mae

#### <span style="color:#55BB99"> 2) Calculate the Precision and Recall for the classification heuristic you made on Sunday:</span>

In [12]:
FP,TP,FN,FN = calc_confuse_rain(rain_pred_df["RAIN"],rain_pred_df["preds"]) # calc confusion matrix

print(f"The Precision for rain predictions are: {precision(TP, FP)}")
print(f"The Recall for rain predictions are: {recall(TP, FN)}")

The Precision for rain predictions are: 1.0
The Recall for rain predictions are: 0.20859362268455647


#### <span style="color:#55BB99"> 3) Calculate the MSE, MAE, or SSE for the regression heuristic you made Monday:</span>

In [13]:
true=prcp_pred_df["PRCP"]
pred=prcp_pred_df["PRCP_mean"]

print(f"The SSE for PRCP predictions are: {rss(true,pred)}")
print(f"The MSE for PRCP predictions are: {mse(true,pred)}")
print(f"The MAE for PRCP predictions are: {mae(true,pred)}")

The SSE for PRCP predictions are: 76.90262920701885
The MSE for PRCP predictions are: 0.0035126583477375806
The MAE for PRCP predictions are: 0.042848794814132896


#### <span style="color:#55BB99"> 4) Cross validation:</span>

In [14]:
# Splitting data

# Split into training and test sets
training, test = train_test_split(
    df, 
    train_size=0.8, # 80% of data to train
    test_size=0.2, # 20% of data to test
    random_state=42
)

In [15]:
# Rain 

# Create Validation Set

# Select target column
target1 = "RAIN"

# feature set "X_train" and "X_test" --> PRCP, TMAX, TMIN
# target set --> Rain 

# Create training and validation sets
X_train1, X_val1, y_train1, y_val1,= train_test_split(
    training.drop([target1, "DATE"], axis=1), # Df with features
    training[target1], # Df with labels
    train_size=0.8,
    test_size=0.2,
    random_state=42
)

lr1 = LogisticRegression() # Create the model

lr1.fit(X_train1, y_train1) # Fit the model

rain_scores = cross_val_score(lr1, X_val1, y_val1, cv=5)
print(f"Average Rain prediction score is: {rain_scores.mean()}")

Average Rain prediction score is: 0.8683929080904829


In [16]:
# PRCP 

# Create Validation Set

# Select target column
target2 = "PRCP"

# feature set "X_train" and "X_test" --> RAIN, TMAX, TMIN
# target set --> PRCP 

# Create training and validation sets
X_train2, X_val2, y_train2, y_val2,= train_test_split(
    training.drop([target2, "DATE"], axis=1), # Df with features
    training[target2], # Df with labels
    train_size=0.8,
    test_size=0.2,
    random_state=42
)

lr2 = LinearRegression()

lr2.fit(X_train2, y_train2) # Fit the model

prcp_scores = cross_val_score(lr2, X_val2, y_val2, cv=5)
print(f"Average PRCP prediction score is: {prcp_scores.mean()}")

Average PRCP prediction score is: 0.5402301550505376
